In [2]:

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization,Dense, Dropout, Flatten, LeakyReLU
from custom_pooling import RMSPooling2D
from keras.optimizers import SGD
from keras_preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score
from keras.callbacks import EarlyStopping, ModelCheckpoint,LearningRateScheduler, TensorBoard
from datetime import datetime as dt
from VCM import VCM 
from keras.initializers import Constant,Orthogonal
from keras import regularizers
#from Kappa_Skl import kappa
import keras.backend as K
import matplotlib.pyplot as plt
import itertools



def get_experiment_id():
    time_str = dt.now().strftime('%Y-%m-%d-%H-%M-%S')
    experiment_id = 'base_{}'.format(time_str)
    return experiment_id

def scheduler(epoch):
    if epoch == 150:
        K.set_value(model.optimizer.lr, 0.0003)
    return K.get_value(model.optimizer.lr)

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
 

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    plt.savefig('CM.png')


model = Sequential()



model.add(Conv2D(32, (5, 5), activation='linear', input_shape=(128, 128, 3), data_format="channels_last", kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))


model.add(Conv2D(32, (3, 3), activation='linear', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))

model.add(MaxPooling2D(pool_size=3, strides=(2, 2)))


model.add(Conv2D(64, (5, 5), activation='linear', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))

model.add(Conv2D(64, (3, 3), activation='linear', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))


model.add(Conv2D(64, (3, 3), activation='linear', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))


model.add(MaxPooling2D(pool_size=3, strides=(2, 2)))
#model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='linear', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))


model.add(Conv2D(128, (3, 3), activation='linear', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))


model.add(Conv2D(128, (3, 3), activation='linear', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))




model.add(RMSPooling2D(pool_size=3,strides=(3, 3)))


model.add(Dropout(0.5))



model.add(Dense(1024, activation='linear', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))

model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())


model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))

model.add(Dense(1, activation='relu', kernel_initializer=Orthogonal(gain=1.0),
                 bias_initializer=Constant(value=0.05),kernel_regularizer=regularizers.l2(0.0005)))

sgd = SGD(lr=0.003, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='mse', optimizer=sgd, metrics=['mae', 'acc'])
model.summary()


experiment_id = get_experiment_id()

def append_ext(fn):
    return fn+".jpeg"


test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255)

traindf=pd.read_csv(("/media/husam/Data/project_2018/train_pre_sep/tran_lab2.csv"), dtype={'image': str, 'level': float})

valdf=pd.read_csv(("/media/husam/Data/project_2018/val_set/val_lab3.csv"), dtype={'image': str, 'level': float})

traindf["image"]=traindf["image"].apply(append_ext)
valdf["image"]=valdf["image"].apply(append_ext)

train_generator = train_datagen.flow_from_dataframe(
dataframe=traindf,
directory=r"/media/husam/Data/project_2018/train_pre_sep2",
x_col="image",
y_col="level",
has_ext=True,                                      
batch_size=64,
seed=42,
shuffle=True,
class_mode="other",
target_size=(128,128))



valid_generator = valid_datagen.flow_from_dataframe(
dataframe=valdf,
directory=r"/media/husam/Data/project_2018/val_set2",
x_col="image",
y_col="level",
has_ext=True,                                      
batch_size=64,
seed=42,
shuffle=True,
class_mode="other",
target_size=(128,128))

test_generator = test_datagen.flow_from_directory(
    directory=r"/media/husam/Data/project_2018/test_pre_sep",
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=1,
    class_mode="sparse",
    shuffle=False,
    seed=42)




STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size





class_weights1 = class_weight.compute_class_weight(
               'balanced',
                np.unique(traindf.level), 
                traindf.level)


callbacks = [
    EarlyStopping(monitor='val_acc', patience=100, min_delta=0.001, verbose=1, restore_best_weights=True),
    ModelCheckpoint(experiment_id + "-val_acc_checkpoint.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
    VCM(valid_generator, STEP_SIZE_VALID),
    LearningRateScheduler(scheduler, verbose=1),
    TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
]



history=model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    class_weight=class_weights1,
                    callbacks=callbacks,
                    epochs=3)


y_pred = model.predict_generator(test_generator, verbose=1)

test_generator.reset()
score = model.evaluate_generator(generator=test_generator, verbose=1)
print('Test loss:', score[0])
print('Test mae:', score[1])
print('Test accuracy:', score[2])





predicted_class_indices = np.round(y_pred).astype(int)
y_true = (test_generator.classes)

print('Confusion Matrix')
cm=confusion_matrix(y_true, predicted_class_indices)
print(cm)

plot_confusion_matrix(cm,target_names=['0', '1', '2', '3', '4'],title='Confusion matrix',normalize=False)
plot_confusion_matrix(cm,target_names=['0', '1', '2', '3', '4'],title='Confusion matrix Normalized',normalize=True)

print('Classification Report')
print(classification_report(y_true, predicted_class_indices, target_names=['0', '1', '2', '3', '4']))


#print('kappa_score')
#print(cohen_kappa_score(y_true, predicted_class_indices))
#
#print('Quadratic_wieghted_kappa')
#print(kappa(y_true, predicted_class_indices, weights='quadratic' ))

# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.savefig('model accuracy.png')
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.savefig('model loss.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 124, 124, 32)      2432      
_________________________________________________________________
batch_normalization_11 (Batc (None, 124, 124, 32)      128       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 122, 122, 32)      9248      
_________________________________________________________________
batch_normalization_12 (Batc (None, 122, 122, 32)      128       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 122, 122, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 60, 60, 32)        0         
__________

ValueError: Found input variables with inconsistent numbers of samples: [79, 5056]